In [2]:
import robotic as ry
import numpy as np
import random
import time
from bridgebuild_env import create_n_boxes
# this import the local manipulation.py .. to be integrated in robotic..
import manipulation as manip
import robotic_execution as robex

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Task Env: Bridge Builder

In [30]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
n_boxes = 3
keep_dist_between = ["box1", "box2", "box3"]
C = create_n_boxes(C = C, N = n_boxes, position = 'fixed')

# remove so collision with wrist camera is ignored
C.delFrame("panda_collCameraWrist")

# for convenience, a few definitions:
qHome = C.getJointState()
gripper = "l_gripper";
palm = "l_palm";
box1 = "box1";
box2 = "box2";
box3 = "box3";
table = "table";
boxSize = C.frame(box1).getSize()

C.view()

0

In [31]:
C.setJointState(qHome)
C.view_raise()
boxes_pos = [C.frame(box1).getPosition(), C.frame(box2).getPosition(), C.frame(box3).getPosition()]
boxes = [box1, box2, box3]
last_box_memory = []
for l in range(3):
        box = np.random.choice(boxes)
        qStart = C.getJointState()
        graspDirection = 'x' #random.choice(['y', 'z']) #'x' not possible: box too large
        placeDirection = 'z' #random.choice(['x', 'y', 'z', 'xNeg', 'yNeg', 'zNeg'])
        info = f'placement {l}: grasp {graspDirection} place {placeDirection}'
        print('===', info)

        M = manip.ManipulationModelling(C, info, helpers=[gripper])
        M.setup_pick_and_place_waypoints(gripper, box, homing_scale=1e-1, accumulated_collisions=True)
        M.grasp_box(1., gripper, box, palm, graspDirection)
        print('place direction:', placeDirection)

        if l > 1:

            offset_xyz = [0.0, 0.0, 0.00]
            poses_from_memory = np.vstack( (C.getFrame(last_box_memory[-2]).getPosition(), C.getFrame(last_box_memory[-1]).getPosition()) )
            middle_between_towers = np.mean(poses_from_memory, 0) + offset_xyz

            dir_vec = C.getFrame(last_box_memory[-2]).getPosition() - C.getFrame(last_box_memory[-1]).getPosition()
            pos2 = C.getFrame(last_box_memory[-2]).getPosition()
            pos1 = C.getFrame(last_box_memory[-1]).getPosition()
            rel_pos =  (pos1 + pos2) / 2 + np.array([0, 0, 0.08])
            
            dir_vec /= np.linalg.norm(dir_vec)

            M.komo.addObjective([2.], ry.FS.position, [box], ry.OT.eq, [1e3], rel_pos)
            M.komo.addObjective([2.], ry.FS.vectorZ, [box], ry.OT.eq, [1e1], dir_vec)
            M.komo.addObjective([2.], ry.FS.vectorZ, [gripper], ry.OT.eq, [1e1], [0, 0, 1])


            
        if l == 1:
            M.place_box(2., box, table, palm, placeDirection)

            M.komo.addObjective([2.], ry.FS.negDistance, [box, last_box_memory[-1]], ry.OT.eq, [1e1], [-.04])
            M.komo.addObjective([2.], ry.FS.scalarProductXY, [box, last_box_memory[-1]], ry.OT.eq, [1e1], [0])
            M.komo.addObjective([2.], ry.FS.positionRel, [box, last_box_memory[-1]], ry.OT.eq, [1.], [.0, .0, .1])
            M.auto_pairwise_keep_dist([], keep_dist_between, margin = 0.08)

        M.no_collision([], palm, table)
        # keep distance between boxes when placed defined by margin
        # x, y, z, smooth - table size: array([2.5 , 2.5 , 0.1 , 0.02])
        # place inital block as chosen by np.random.choice at a random location
        if l == 0:
            M.place_box(2., box, table, palm, placeDirection)

            M.target_relative_xy_position(2., box, table, [np.random.uniform(0.0, 0.3), np.random.uniform(0.0, 0.3)])
            M.auto_pairwise_keep_dist([], keep_dist_between, margin = 0.08)


        ways = M.solve()
        
        if not M.feasible:
            continue

        M2 = M.sub_motion(0)
        # M = manip.ManipulationModelling(C, info, helpers=[gripper])
        # M.setup_point_to_point_motion(qStart, ways[0])
        M2.no_collision([.3,.7], palm, box)
        
        M2.retract([.0, .2], gripper, dist = .13)
        M2.approach([.8, 1.], gripper)
        M2.solve()
        if not M2.feasible:
            continue

        M3 = M.sub_motion(1)
        #manip.ManipulationModelling(C, info)
        # M.setup_point_to_point_motion(ways[0], ways[1])
        M3.no_collision([], table, box)

        M3.solve()
        if not M3.ret.feasible:
            continue

        M2.play(C)
        C.attach(gripper, box)
        M3.play(C)
        C.attach(table, box)
        # update memory
        last_box_memory.append(box)
        # remove box that was previously used
        boxes.remove(box)
        # update positions
        boxes_pos = [C.frame(box1).getPosition(), C.frame(box2).getPosition(), C.frame(box3).getPosition()]


del M

=== placement 0: grasp x place z
place direction: z
  -- feasible:placement 0: grasp x place z
     { time: 0.031691, evals: 57, done: 1, feasible: 1, sos: 0.076897, f: 0, ineq: 4.57125e-08, eq: 0.0264871 }
  -- feasible:sub_motion_0--placement 0: grasp x place z
     { time: 0.056169, evals: 24, done: 1, feasible: 1, sos: 2.83893, f: 0, ineq: 1.13107e-07, eq: 0.475345 }
  -- feasible:sub_motion_1--placement 0: grasp x place z
     { time: 0.106763, evals: 43, done: 1, feasible: 1, sos: 0.187854, f: 0, ineq: 0, eq: 0.323681 }
=== placement 1: grasp x place z
place direction: z
  -- feasible:placement 1: grasp x place z
     { time: 0.027477, evals: 53, done: 1, feasible: 1, sos: 0.0828, f: 0, ineq: 0.210057, eq: 0.472798 }
  -- feasible:sub_motion_0--placement 1: grasp x place z
     { time: 0.080982, evals: 27, done: 1, feasible: 1, sos: 1.81618, f: 0, ineq: 3.74534e-07, eq: 0.317016 }
  -- feasible:sub_motion_1--placement 1: grasp x place z
     { time: 0.486122, evals: 175, done: 1,

### On real robot: BotOp

In [32]:
import robotic as ry
import numpy as np
import random
import time
from bridgebuild_env import create_n_boxes
# this import the local manipulation.py .. to be integrated in robotic..
import manipulation as manip
import robotic_execution as robex

In [33]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
n_boxes = 3
keep_dist_between = ["box1", "box2", "box3"]
C = create_n_boxes(C = C, N = n_boxes, position = 'fixed')

# remove so collision with wrist camera is ignored
C.delFrame("panda_collCameraWrist")

# for convenience, a few definitions:
qHome = C.getJointState()
gripper = "l_gripper";
palm = "l_palm";
box1 = "box1";
box2 = "box2";
box3 = "box3";
table = "table";
boxSize = C.frame(box1).getSize()

C.view()

0

In [34]:
C.view()
bot = robex.Robot(C, on_real=False)
bot.goHome(C)

-- bot.cpp:~BotOp:128(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:56(0) shutting down SimThread
-- simulation.cpp:~Simulation:149(0) shutting down Simulation


In [35]:
C.setJointState(qHome)
C.view_raise()
boxes_pos = [C.frame(box1).getPosition(), C.frame(box2).getPosition(), C.frame(box3).getPosition()]
boxes = [box1, box2, box3]
last_box_memory = []
paths = []
vis = 1
for l in range(3):
        box = np.random.choice(boxes)
        qStart = C.getJointState()
        graspDirection = 'x' #random.choice(['y', 'z']) #'x' not possible: box too large
        placeDirection = 'z' #random.choice(['x', 'y', 'z', 'xNeg', 'yNeg', 'zNeg'])
        info = f'placement {l}: grasp {graspDirection} place {placeDirection}'
        print('===', info)

        M = manip.ManipulationModelling(C, info, helpers=[gripper])
        M.setup_pick_and_place_waypoints(gripper, box, homing_scale=1e-1, accumulated_collisions=True)
        M.grasp_box(1., gripper, box, palm, graspDirection)
        print('place direction:', placeDirection)

        if l > 1:

            dir_vec = C.getFrame(last_box_memory[-2]).getPosition() - C.getFrame(last_box_memory[-1]).getPosition()
            pos2 = C.getFrame(last_box_memory[-2]).getPosition()
            pos1 = C.getFrame(last_box_memory[-1]).getPosition()
            rel_pos =  (pos1 + pos2) / 2 + np.array([0, 0, 0.12])
            
            dir_vec /= np.linalg.norm(dir_vec)

            M.komo.addObjective([2.], ry.FS.position, [box], ry.OT.eq, [1e3], rel_pos)
            M.komo.addObjective([2.], ry.FS.vectorZ, [box], ry.OT.eq, [1e1], dir_vec)
            M.komo.addObjective([2.], ry.FS.vectorZ, [gripper], ry.OT.eq, [1e1], [0, 0, 1])


            
        if l == 1:
            M.place_box(2., box, table, palm, placeDirection)

            M.komo.addObjective([2.], ry.FS.negDistance, [box, last_box_memory[-1]], ry.OT.eq, [1e1], [-.04])
            M.komo.addObjective([2.], ry.FS.scalarProductXY, [box, last_box_memory[-1]], ry.OT.eq, [1e1], [0])
            M.komo.addObjective([2.], ry.FS.positionRel, [box, last_box_memory[-1]], ry.OT.eq, [1.], [.0, .0, .1])
            M.auto_pairwise_keep_dist([], keep_dist_between, margin = 0.08)

        M.no_collision([], palm, table)
        # keep distance between boxes when placed defined by margin
        # x, y, z, smooth - table size: array([2.5 , 2.5 , 0.1 , 0.02])
        # place inital block as chosen by np.random.choice at a random location
        if l == 0:
            M.place_box(2., box, table, palm, placeDirection)

            M.target_relative_xy_position(2., box, table, [np.random.uniform(0.0, 0.3), np.random.uniform(0.0, 0.3)])
            M.auto_pairwise_keep_dist([], keep_dist_between, margin = 0.08)


        ways = M.solve()

        if not M.feasible:
            continue

        M2 = M.sub_motion(0)
        # M = manip.ManipulationModelling(C, info, helpers=[gripper])
        # M.setup_point_to_point_motion(qStart, ways[0])
        M2.no_collision([.3,.7], palm, box)
        
        M2.retract([.0, .2], gripper, dist = .13)
        M2.approach([.8, 1.], gripper)
        paths.append(M2.solve())
        
        if not M2.feasible:
            continue

        M3 = M.sub_motion(1)
        #manip.ManipulationModelling(C, info)
        # M.setup_point_to_point_motion(ways[0], ways[1])
        M3.no_collision([], table, box)

        paths.append(M3.solve())
        if not M3.ret.feasible:
            continue

        if vis == 0:
            M2.play(C)
            C.attach(gripper, box)
            M3.play(C)
            C.attach(table, box)
        bot.execute_path_blocking(C, path = paths[-2])
        bot.grasp(C)

        bot.execute_path_blocking(C, path = paths[-1])
        bot.release(C)
        # update memory
        last_box_memory.append(box)
        # remove box that was previously used
        boxes.remove(box)
        # update positions
        boxes_pos = [C.frame(box1).getPosition(), C.frame(box2).getPosition(), C.frame(box3).getPosition()]
        
del M

=== placement 0: grasp x place z
place direction: z
  -- feasible:placement 0: grasp x place z
     { time: 0.01414, evals: 22, done: 1, feasible: 1, sos: 0.095614, f: 0, ineq: 7.00017e-10, eq: 0.00100033 }
  -- feasible:sub_motion_0--placement 0: grasp x place z
     { time: 0.057229, evals: 21, done: 1, feasible: 1, sos: 3.01092, f: 0, ineq: 2.44944e-07, eq: 0.0166191 }
  -- feasible:sub_motion_1--placement 0: grasp x place z
     { time: 0.224025, evals: 81, done: 1, feasible: 1, sos: 0.129811, f: 0, ineq: 0.000808956, eq: 0.0161285 }
-- kin_physx.cpp:addJoint:299(0) ADDING JOINT l_panda_joint7-box2 of type rigid with rel [0, 0, 0]
-- kin_physx.cpp:removeJoint:893(0) REMOVING JOINT 0x4277de0-0x56721b0 of type rigid
=== placement 1: grasp x place z
place direction: z
  -- feasible:placement 1: grasp x place z
     { time: 0.030207, evals: 65, done: 1, feasible: 1, sos: 0.0879588, f: 0, ineq: 0.206706, eq: 0.422978 }
  -- feasible:sub_motion_0--placement 1: grasp x place z
     { time